In [47]:
# -*- coding: UTF-8 -*-
# CGI处理模块
import cgi, cgitb
# 创建 FieldStorage 的实例化
form = cgi.FieldStorage() 

# 获取数据

days_input = form.getvalue('daysx')

print("Content-type:text/html")
print()


from datetime import datetime,timedelta
from ftplib import FTP
import pandas as pd

enddate=datetime.now()
#结束日期

if days_input == None:
    days=10
else:
    days=days_input
#days 为计算天数

startdate=enddate-timedelta(days=days)
#开始日期
dates=[]
#结束与开始日期之间的日期列表
date=startdate
while date<=enddate:
    if date.isoweekday()<6:
        #isoweekday,周一等于1，周日等于7
        dates.append(date.strftime('%Y%m%d'))
    date=date+timedelta(1)
print(dates)

ftp = FTP('106.12.3.198')     # connect to host, default port
ftp.login()                     # user anonymous, passwd anonymous@
ftp.cwd('new_tdx\T0002\export')               # change  directory
ftp.encoding='gbk'

['20191225', '20191226', '20191227', '20191230', '20191231', '20200101', '20200102', '20200103']


In [48]:
filenamelist=[]
#读取出来的文件名存在此列表
for date in dates:
    filename="板块指数"+date+".txt"
    try:
        with open(filename, 'wb') as fp:
            ftpcommand="RETR "+filename
            ftp.retrbinary(ftpcommand, fp.write)
    except Exception as e:
        print("读取文件:",filename,"发生错误：",e)
    else:
        filenamelist.append(filename)
ftp.quit()


读取文件: 板块指数20191225.txt 发生错误： 550 The system cannot find the file specified. 
读取文件: 板块指数20191226.txt 发生错误： 550 The system cannot find the file specified. 
读取文件: 板块指数20191227.txt 发生错误： 550 The system cannot find the file specified. 
读取文件: 板块指数20191230.txt 发生错误： 550 The system cannot find the file specified. 
读取文件: 板块指数20191231.txt 发生错误： 550 The system cannot find the file specified. 
读取文件: 板块指数20200101.txt 发生错误： 550 The system cannot find the file specified. 


'221 Goodbye.'

In [49]:
#用命名空间自动生成变量名，将读出的文件存在df中，变量名从df1开始   
i=1
df=[]
for file in filenamelist:
    with open(file,'r',encoding="gbk") as f:
        a=f.readlines()
        data=[]
        for j in a:
            data.append(j[:-1].split('\t')[:-1])
        locals()["df"+str(i)]=pd.DataFrame(data[1:-1],columns=data[0])
        #j[:-1].split('\t')[:-1],第一个[:-1]去掉\t\n,第二个[:-1]去掉最后一个空项目，剩下都是列名
        df.append(eval("df"+str(i)))
        i=i+1
    
        


In [50]:
#对于df中每个DATAFRAME按照涨停数排序，取前20个，然后存入dftemp列表中
dftemp=[]
for temp in df:
    temp[['涨停数']] = temp[['涨停数']].astype(int)
    #强制转换为Int型，然后按此列排序
    locals()["df"+str(i)]=temp.sort_values(by="涨停数" , ascending=False)[:20]
    dftemp.append(eval("df"+str(i)))


In [42]:
'''
#直接通过CGI输出，不输出html文件
HEADER = '''
    <html>
        <head>
            <meta charset="UTF-8">
            <style>
            .industry {
            margin:0 auto
            }
            </style>
        </head>
        <body>
    '''
FOOTER = '''
        </body>
    </html>
    '''
 
with open('a.html', 'w',encoding='utf-8') as f:
    i=0
    f.write(HEADER)
    for df in dftemp:
        f.write(filenamelist[i])
        i=i+1
        f.write(df.iloc[:20,:14].to_html(classes='industry'))
    f.write(FOOTER)
        '''

In [87]:
print(HEADER)
i=0
for dfx in dftemp:
    print(filenamelist[i])
    i=i+1
    print("<table class='industry'>")
    df=dfx.iloc[:20,:14]
    
    print("<tr>")
    for x in df.columns:
        print("<th>"+x+"</th>")
    print("</tr>")
    #第一行标题
    
    #下面打印出每行数据内容
    for x in range(len(df)):
        print("<tr>")
        for y in df.iloc[x]:
            print("<td>"+str(y)+"</td>")
        print("</tr>")
            
    print("</table>")
print(FOOTER)


    <html>
        <head>
            <meta charset="UTF-8">
            <style>
            .industry {
            margin:0 auto
            }
            </style>
        </head>
        <body>
    
板块指数20200102.txt
<table class='industry'>
<tr>
<th>代码</th>
<th>名称</th>
<th>涨幅%</th>
<th>现价</th>
<th>涨跌</th>
<th>涨速%</th>
<th>量比</th>
<th>涨跌数</th>
<th>涨停数</th>
<th>总金额</th>
<th>开盘金额</th>
<th>换手%</th>
<th>连涨天</th>
<th>3日涨幅%</th>
</tr>
<tr>
<td>880791</td>
<td>网红经济</td>
<td>7.20</td>
<td>1079.59</td>
<td>72.53</td>
<td>0.06</td>
<td>  1.29</td>
<td>40,0</td>
<td>18</td>
<td>13879355392</td>
<td>231216896</td>
<td>          5.90</td>
<td>4</td>
<td>7.87</td>
</tr>
<tr>
<td>880494</td>
<td>互联网</td>
<td>5.13</td>
<td>2714.03</td>
<td>132.36</td>
<td>0.02</td>
<td>  1.28</td>
<td>69,0</td>
<td>13</td>
<td>23274711040</td>
<td>157177200</td>
<td>          5.22</td>
<td>2</td>
<td>5.00</td>
</tr>
<tr>
<td>880590</td>
<td>网络游戏</td>
<td>5.08</td>
<td>1313.67</td>
<td>63.51</td>
<td>0.04</td>
<td> 

880516
ST板块
0.85
873.34
7.34
0.01
  0.98
94,26
22
4309850112
66126000
          1.20
3
2.60
880879
低价股
1.04
833.44
8.60
0.05
  1.13
116,43
18
14121149440
143488000
          1.20
3
3.13
880946
区块链
1.37
801.96
10.80
0.14
  1.26
147,45
14
66888081408
579379584
          3.53
4
4.42
880880
近期强势
0.47
432.06
2.02
0.11
  1.36
41,51
12
55430197248
965641792
          7.21
3
4.57
880868
高贝塔值
0.69
975.03
6.69
0.04
  1.23
55,41
12
55986331648
935353600
          5.75
4
3.63
880207
北京板块
0.37
1322.96
4.88
0.11
  1.12
209,116
11
87663984640
765908928
          0.48
4
2.02
880494
互联网
2.36
2778.19
64.16
0.07
  1.50
54,15
11
29510113280
430636480
          6.34
3
7.78
880892
高质押股
0.46
650.58
2.97
0.03
  1.11
105,78
11
22695645184
231017200
          1.73
4
2.61
880360
农林牧渔
0.96
1686.06
16.09
0.14
  1.58
49,19
11
18872571904
383819808
          2.97
5
6.26
880791
网红经济
1.57
1096.57
16.98
0.08
  1.69
27,19
10
21704837120
494543712
          6.87
5
9.45
880710
种业
9.47
1125.81
97.37
0.28
  2.24
11,0
9
5196

In [67]:
a.columns[13]

'3日涨幅%'